In [1]:
from openpyxl import load_workbook
import subprocess
import pandas as pd
import datetime

In [3]:
# Necessary functions
def update_calibration_parameters(sheet, parameter, new_value):
    '''
    This function updates the calibration sheet in DIGNAD.
    '''

    for row in sheet.iter_rows():
        for cell in row:
            if cell.value == parameter:
                # Assuming the value needs to be updated in the cell right after the parameter
                target_cell = sheet.cell(row=cell.row, column=cell.column + 1)
                try:
                    # Convert to float first
                    target_cell.value = float(new_value)
                except ValueError:
                    # if above doesn't work, save as string
                    target_cell.value = new_value
                return True  # Return after the first match to avoid unnecessary updates
    return False  # Return False if parameter not found

def update_natural_hazard_parameters(nat_disaster_year, recovery_period, tradable_impact,
                                    nontradable_impact, reconstruction_efficiency, public_debt_premium,
                                    public_impact, private_impact, share_tradable):
    '''
    This function returns a dictionary that will be used to populate the Disasters sheet.
    It takes as input the 9 parameters the user typically has to set in the Disasters sheet.
    '''

    excel_updates = {
    (3, 4): nat_disaster_year - 2021, # C4 cell update
    (4, 4): nat_disaster_year, # D4 cell update
    (4, 7): tradable_impact, # D7 cell update
    (4, 8): nontradable_impact, # D8 cell update
    (4, 9): reconstruction_efficiency, # D9 cell update
    (4, 10): public_debt_premium, # D10 cell update
    (4, 11): public_impact, # D11 cell update
    (4, 12): private_impact, # D12 cell update
    (4, 13): share_tradable, # D13 cell update
    (3, 17): nat_disaster_year, # C17 cell update
    (4, 17): nat_disaster_year, # D17 cell update
    (3, 18): nat_disaster_year + recovery_period, # C18 cell update
    (4, 18): nat_disaster_year + recovery_period, # D18 cell update
    (3, 20): nat_disaster_year, # C20 cell update
    (3, 21): nat_disaster_year + recovery_period, # C21 cell update
    (3, 23): nat_disaster_year, # C23 cell update
    (3, 24): nat_disaster_year + recovery_period, # C24 cell update
    (3, 26): nat_disaster_year, # C26 cell update
    (3, 27): nat_disaster_year + recovery_period # C27 cell update
    }

    return excel_updates

In [2]:
def run_DIGNAD(calibration_csv, nat_disaster_year, recovery_period, tradable_impact, nontradable_impact,
                reconstruction_efficiency, public_debt_premium, public_impact, private_impact, share_tradable):
    '''
    This function runs on instance of DIGNAD with a prespecified calibration csv.
    Parameters passed to the function are the natural hazard parameters.
    Function outputs a list of GDP losses - from 2021 - 2040
    '''

    ### 1. Load the original Excel file - this is where all DIGNAD parameters are set
    excel_file = r"D:\projects\sovereign-risk\Thailand\DIGNAD\DIGNAD_Toolkit_2023\PW_SHARED_2023\DIGNAD_Toolkit\DIGNAD_python\input_DIG-ND.xlsx"
    wb = load_workbook(excel_file)

    ### 2. Load the CSV with calibration parameters
    calibration_df = pd.read_csv(calibration_csv)

    ### 3. Set calibration parameters
    sheet = wb['Calibration']
    # Iterate over the calibration DataFrame rows
    for index, row in calibration_df.iterrows():
        parameter = row['Parameters']  # The column name in your CSV for the parameter names
        new_value = row['Values']       # The column name in your CSV for the new values
        updated = update_calibration_parameters(sheet, parameter, new_value)
        if not updated:
            print(f"Parameter '{parameter}' not found in the Excel sheet.")

    ### 4. Update disasters sheet
    natdisaster_params = update_natural_hazard_parameters(nat_disaster_year, recovery_period, tradable_impact,
                                                                nontradable_impact, reconstruction_efficiency,
                                                                public_debt_premium, public_impact, private_impact, share_tradable)
    sheet = wb['Disasters']
    for (col, row), value in natdisaster_params.items():
        cell = sheet.cell(row=row, column=col)
        cell.value = value

    ### 5. Save Excel Workbook
    wb.save(excel_file)

    ### 6. Run Matlab
    matlab_script = r"D:\projects\sovereign-risk\Thailand\DIGNAD\DIGNAD_Toolkit_2023\PW_SHARED_2023\DIGNAD_Toolkit\DIGNAD_python\simulate.m"
    result = subprocess.call(["matlab", "-batch", "run('" + matlab_script + "')"])
    if int(result) != 0:
        print("MATLAB script not executed succesfully")
        return None, None

    ### 7. Read results from Excel sheet
    # Get today's date as that is the name of file and directory
    today = datetime.datetime.today().strftime("%d%b%Y")
    file_path = r"D:\projects\sovereign-risk\Thailand\DIGNAD\DIGNAD_Toolkit_2023\PW_SHARED_2023\DIGNAD_Toolkit\DIGNAD_python\Excel output\%s\Model_output_%s.xlsx" % (today, today)
    df = pd.read_excel(file_path)
    years = list(df.iloc[:, 0])
    gdp_impact = list(df.iloc[:, 1])

    return gdp_impact, years
    

In [8]:
# Run DIGNAD
# Function inputs
calibration_csv = r"D:\projects\sovereign-risk\Thailand\data\DIGNAD\inputs\THA_2022_calibration_revised.csv"
nat_disaster_year = 2022
recovery_period = 5 # years
tradable_impact = 0.004
nontradable_impact = 0.004
reconstruction_efficiency = 0
public_debt_premium = 0
public_impact = 0.004
private_impact = 0.004
share_tradable = 0.5
# Run function
gdp_impact, years = run_DIGNAD(calibration_csv, nat_disaster_year, recovery_period, tradable_impact, nontradable_impact, reconstruction_efficiency,
                                public_debt_premium, public_impact, private_impact, share_tradable)

MATLAB script not executed succesfully


In [7]:
gdp_impact

[0.0,
 -0.2926724603718345,
 -0.2355764924967918,
 -0.1470612400606197,
 -0.09273884367914098,
 -0.059541008482133506,
 -0.026884445481267782,
 -0.011555401655127806,
 -0.010266076844889227,
 -0.009156639108964804,
 -0.008160562244241554,
 -0.007277393012516953,
 -0.006491111363171598,
 -0.005791710619884594,
 -0.005169211637023885,
 -0.0046150721877746825,
 -0.004121642312082674,
 -0.003682161889262403,
 -0.0032906308157665265,
 -0.0029417286179223012]

In [17]:
gdp_impact

[0.0,
 -0.29349361194114243,
 -0.2782128866824718,
 -0.20265985385069518,
 -0.15467000269849818,
 -0.12364571451882744,
 -0.09033360967221071,
 -0.0733523658258517,
 -0.06876619387456984,
 -0.06449328742601512,
 -0.060490511015443005,
 -0.056740538372968,
 -0.05322667350209365,
 -0.04993344363558583,
 -0.04684647844703793,
 -0.04395241987450449,
 -0.04123884034893388,
 -0.038694168960551334,
 -0.03630762462925041,
 -0.03406915548666012]